Importing OpenCV and Tesseract libraries.

In [1]:
import cv2
import pytesseract
import numpy as np
import regex as re

pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

In [2]:
import pkg_resources
pkg_resources.working_set.by_key['pytesseract'].version

'0.3.10'

This can extract a cartboard box from image using visual processing. Only works when box is moving on a different color conveyor. Uses color separation.

In [3]:
img = cv2.imread('package.jpg')
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
th = cv2.threshold(hsv[:,:,0],127,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)[1]

def get_region(image):
    contours, hierarchy = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    c = max(contours, key = cv2.contourArea)
    black = np.zeros((image.shape[0], image.shape[1]), np.uint8)
    mask = cv2.drawContours(black,[c],0,255, -1)
    return mask

mask = get_region(th)

masked_img = cv2.bitwise_and(img, img, mask = mask)

cv2.imwrite("masked_image.jpg", masked_img)

True

Transforming image using OpenCV and extracting numbers using Tesseract. The image is blurred, thresholded and diluted. Then Tesseract extracts test with flags to only search for digits.

In [4]:
def getText2(name, fileIsImage=False):
    # If fileIsImage = False, reads image from file, otherwise assumes image is already given
    if(not fileIsImage):
        img = cv2.imread(name)
    else:
        img = name
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #noise removal
    noise=cv2.medianBlur(gray,3)
    # thresholding# converting it to binary image by Thresholding
    # this step is require if you have colored image because if you skip this part
    # then tesseract won’t able to detect text correctly and this will give incorrect #result
    thresh = cv2.threshold(noise, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    # thresh = cv2.bitwise_not(thresh)
    cv2.imwrite("transformed.png", thresh)
    # Using tesseract to extract numbers
    # txt = pytesseract.image_to_string(thresh)
    txt = pytesseract.image_to_string(thresh, config="--psm 6 digits")
    return txt

FACT: vid_1 has 11 boxes captured

In [5]:
def getNumber(image):
    # grayscale
    result = image.copy()
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)


    # adaptive threshold
    thresh = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,51,9)


    # Fill rectangular contours
    cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        cv2.drawContours(thresh, [c], -1, (255,255,255), -1)

    # Morph open
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9,9))
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=4)


    # Draw rectangles, the 'area_treshold' value was determined empirically
    cnts = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    area_treshold = 4000
    for i,c in enumerate(cnts):
        if cv2.contourArea(c) > area_treshold :
            x,y,w,h = cv2.boundingRect(c)
            cropimage=image[y:y+h,x:x+w]
            cv2.imwrite(f'realImage{i}.jpg',cropimage)
            text = getText2(cropimage, True)
            if re.search('[0-9]+\.[0-9]+\.[0-9]+',text):
                return text.rstrip()

In [ ]:
path = "vid_1.MOV"
VALIDCODE='504.832.89'
vidcap = cv2.VideoCapture(path)
rateReduction = 40
framenbr = 0
success,frame = vidcap.read()
count = 0

numberList=list()
boxList=['1.1.1', '1.1.1', '1.1.1']
resultList=list()

while success:
    success, frame = vidcap.read()
    if not success:
        break
    # cv2.imwrite("frame%d.png" % framenbr, frame) 
    num = getNumber(frame)
    # list with all numbers
    numberList.append(num)
    # find results
    if num is None and boxList is not None:
        result = max(set(boxList), key =boxList.count)
        resultList.append(result)
        if result != VALIDCODE:
            print(f"ERROR: {VALIDCODE} expected, {result} found!!!")
        boxList = None
    elif num is not None:
        if boxList is None:
            boxList=list()
        boxList.append(num)
    count += rateReduction # i.e. at 30 fps, this advances one second
    vidcap.set(cv2.CAP_PROP_POS_FRAMES, count)
    # print(text)
    framenbr += 1

In [ ]:
resultList

In [ ]:
#load the image
image = cv2.imread('frame10.png')

# grayscale
result = image.copy()
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
cv2.imwrite("testGray.jpg", gray) 

# adaptive threshold
thresh = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,51,9)
cv2.imwrite("testhresh.jpg", thresh) 

# Fill rectangular contours
cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    cv2.drawContours(thresh, [c], -1, (255,255,255), -1)

# Morph open
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9,9))
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=4)
cv2.imwrite("testopen.jpg", opening)

# Draw rectangles, the 'area_treshold' value was determined empirically
cnts = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
area_treshold = 4000
for c in cnts:
    if cv2.contourArea(c) > area_treshold :
      x,y,w,h = cv2.boundingRect(c)
      cv2.rectangle(image, (x, y), (x + w, y + h), (36,255,12), 3)

cv2.imshow('thresh', thresh)
cv2.imshow('opening', opening)
cv2.imshow('image', image)
cv2.waitKey()

0

: 